## Objectifs
Ce notebook est dedié à la mise en forme des données issues de la simulation SUMO afin de pouvoir être utilisées par les GNN (réseaux neuronaux sur graphs). L'objectif est de simuler la présence de détécteur moyenneurs de vitesse placés uniformémént sur le circuit routier.

## Remarques sur les données
Les données vierges provenant tout juste de la simulation sont en format JSON. Elles sont hiérarchisées selon l'indice de l'ittération suivi de l'indice de la voiture en question puis d'un triplet (vitesse, (abscisse, ordonnée)). Le problème de ce dernier triplet de coordonnées est leur origine. Lors de l'obtention de ces dernières SUMO utilise une origine arbitraire ce qui nous oblige à définir notre propre origine pour être en cohérence avec les coordonées de notre système.

## Définition et placement des détécteurs
Afin d'obtenir la vitesse moyenne des véhicules dans une zone spécifique, il faut compter les véhicules présents dans cette zonne, additionner leur vitesses puis calculer la vitesse moyenne. Comme précisé dans la séction "Ojectifs" les détécteurs doivent être placés uniformément sur le circuit routier. Pour cela on définit une longeur de secteur et un pas. On parcours l'entierté de la carte en sotant à chaque étape le pas définit puis si il y a assez de route présente dans le sécteur en question, on le garde. Ceci est achevé par la fonction **get_sectors**. Pour obtenir la vitesse moyenne des véhicules présent dans un sécteur, on a mis en place la fonction **get_sector_data** qui utilise à son tour la fonctoin **car_in_sector**


In [2]:
import sys
import json
import numpy as np
import pickle
from PIL import Image
import matplotlib.pyplot as plt

In [3]:
def remove_empty_timesteps(f) :
	raw_data = json.load(f)
	data = {k:v for k,v in raw_data.items() if v}

	json_object = json.dumps(data, indent=4)

	with open("processed.json", "w") as outfile:
		outfile.write(json_object)

	return data

In [4]:
def json_to_list(data) :
	timesteps = []
	for k,v in data.items() :
		timestep = []
		for k_car, car in v.items() :
			timestep.append((car[0], car[1]))
		if len(timestep) > 0 :
			timesteps.append(timestep)
	return timesteps

In [5]:
def trace_routes(data, minc, shape = (200,200)) :
	route_map = np.zeros((int(np.floor(shape[0])), int(np.floor(shape[1]))))
	centered_data = []
	print("Starting route tracing.....")
	for ts in data :
		centered_ts = []
		for car in ts :
			centered_ts.append((car[0], (car[1][0]-minc[0], car[1][1]-minc[1])))
			int_x = int(np.floor(car[1][0]-minc[0]))
			int_y = int(np.floor(car[1][1]-minc[1]))
			route_map[int_x][int_y+1] = 1
			route_map[int_x][int_y] = 1
			route_map[int_x][int_y-1] = 1
			route_map[int_x+1][int_y+1] = 1
			route_map[int_x+1][int_y] = 1
			route_map[int_x+1][int_y-1] = 1
			route_map[int_x-1][int_y+1] = 1
			route_map[int_x-1][int_y] = 1
			route_map[int_x-1][int_y-1] = 1
		centered_data.append(centered_ts)

	return route_map, centered_data

In [6]:
def car_in_sector(coord, x, y, sector_length) :
	if abs(coord[0] - x) <= sector_length and abs(coord[1] - y) <= sector_length :
			return True
	return False

In [7]:
def draw_car(sector, car) :	
	int_x = int(car[1][0])
	int_y = int(car[1][1])  
	if int_x + 1 == sector.shape[0] or int_y + 1 == sector.shape[1] :
		sector[int_x][int_y] = 230
	else :
		sector[int_x][int_y+1] = 230
		sector[int_x][int_y] = 230
		sector[int_x][int_y-1] = 230
		sector[int_x+1][int_y+1] = 230
		sector[int_x+1][int_y] = 230
		sector[int_x+1][int_y-1] = 230
		sector[int_x-1][int_y+1] = 230
		sector[int_x-1][int_y] = 230
		sector[int_x-1][int_y-1] = 230
	return sector

In [8]:
def draw_sectors(sector_map, x, y, sector_length, car=False) :
	for i in range(-sector_length, sector_length):
		for j in range(-sector_length, sector_length):
			if not(i == -sector_length or j == -sector_length or i == sector_length-1 or j == sector_length-1) :
				if car :
					sector_map[x+i,y+j] = 50
				else :
					sector_map[x+i,y+j] = 170
			else : 
				sector_map[x+i,y+j] = 255
	return sector_map

In [9]:
def get_sectors(route_map, sector_length, gap_x, gap_y) :
	horizontal = (route_map.shape[0]-2*sector_length) // (gap_x)
	vertical = (route_map.shape[1]-2*sector_length) // (gap_y)
	sector_map = np.zeros_like(route_map) 
	sectors = []
	s_x = 0; s_y = 0;
	for i in range(horizontal) :
		for j in range(vertical) :
			x = (sector_length+gap_x)*i; y = (sector_length+gap_y)*j
			sector = route_map[-sector_length+x:sector_length+x, -sector_length+y:sector_length+y]

			if np.count_nonzero(sector)> 4*sector_length**2*0.1:
				sectors.append((x,y))
				sector_map = draw_sectors(sector_map, x, y, sector_length)

	route_map = np.multiply(sector_map, route_map)
	img = Image.fromarray(sector_map)
	img.save('images/sector_map.png')
	return sectors, sector_map

In [24]:
def get_sector_data(ts, sectors, sector_length):
	average_speed = np.zeros((len(sectors),))
	cars_in_sector = np.zeros((len(sectors),))
	detected = 0
	for idx in range(len(sectors)) :
		for car in ts :
			if car_in_sector(car[1], sectors[idx][0], sectors[idx][1], sector_length):
				average_speed[idx] += car[0]
				cars_in_sector[idx] += 1
				detected += 1
	print(f"Detected {detected} out of {len(ts)} cars !")
    cars_in_sector = cars_in_sector-1* np.ones_like(cars_in_sector[cars_in_sector])
	average_speed = np.divide(average_speed, cars_in_sector)
	return average_speed

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 12)

In [11]:
def color_sectors(ts, sector_map, sectors, sector_length):
	for idx in range(len(sectors)) :	
		for car in ts :
			if car_in_sector(car[1], sectors[idx][0], sectors[idx][1], sector_length):
				sector_map = draw_sectors(sector_map, sectors[idx][0], sectors[idx][1], sector_length, car = True)
			else:
			 	sector_map = draw_car(sector_map, car)
	img = Image.fromarray(sector_map)
	img.save('images/car_sector_map.png')

In [12]:
f = open(f'data.json')
minc = (-53.11,2)
data = remove_empty_timesteps(f)
list_data = json_to_list(data)
rmap, centered_data = trace_routes(list_data, minc = minc)
rmap= (rmap* 255).astype(np.uint8)
img = Image.fromarray(rmap)
img.save('images/traced_routes.png')
img.show()
sector_length = 7
step = 7
sectors, sector_map = get_sectors(rmap, sector_length, step, step)

Starting route tracing.....


In [13]:
def dist(sec1, sec2):
    return np.sqrt((sec1[0]-sec2[0])**2 + (sec1[1]-sec2[1])**2)

def get_graph(sectors, sector_length, step) :
    graph = np.zeros((len(sectors), len(sectors)))
    for i, sector in enumerate(sectors) :
        for j, neigh in enumerate(sectors) :
            if dist(sector, neigh) < step :
                graph[i][j] = 1
    
    return graph

In [14]:
print(sectors)

[(28, 14), (42, 14), (56, 14), (70, 14), (70, 28), (84, 28), (84, 42), (84, 56), (84, 70), (84, 84), (84, 98), (84, 112), (98, 28), (112, 28), (126, 14), (140, 14), (154, 14), (168, 14)]


In [15]:
get_graph(sectors, sector_length, step)

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0

In [17]:
img = color_sectors(centered_data[30], sector_map, sectors, sector_length)
print(get_sector_data(centered_data, sectors, sector_length))

Detected 16 out of 21 cars !
[0.         0.         0.         0.         0.         0.
 3.79030785 5.48676463 0.         5.70830456 0.         5.94013526
 3.09586938 4.51894798 0.         0.         6.33045009 0.        ]


In [18]:
def get_data(centered_data, sectors, sector_length):
    X = []
    Y = []
    for data in centered_data : 
        v_avg = get_sector_data(centered_data[30], sectors, sector_length)
        X.append(v_avg)
        Y.append(v_avg[v_avg<3])
        

IndexError: list index out of range